In [43]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline

## Рестораны Петербурга

In [44]:
#feel free to change the folder name 
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab and amounting drive')
    from google.colab import drive
    drive.mount('/content/gdrive')
    Path("/content/gdrive/My Drive/Smolny/QuantMethods").mkdir(parents=True, exist_ok=True) 
    os.chdir("/content/gdrive/My Drive/Smolny/QuantMethods") 
else:
      print('Not running on CoLab')

Not running on CoLab


In [219]:
#Загрузим файл с гитхаба, если его нет
#!curl -OL https://raw.githubusercontent.com/iknyazeva/ML2020/master/assignments/datasets/RestAll.txt
#!ls

In [46]:
def read_rests_data(filepath):
    """
    Function for reading txt data to dataframe
    :param filepath: (str) path to file
    :return df: pandas data frame
    """

    with open(filepath) as f:
        data = f.read().split('\n')

    bar_dict=dict()

    for line in data:

        fields = [re.split("(?:(?<=url)|(?<=title)|(?<=language)|(?<=tag)):",x) for x in line.strip().split("\t")]

        if (len(fields) < 2) or ('offer' in fields[0][1]):
            continue
        bar_descr = dict()

        for name, value in fields:
            if name == 'title':
                bar_title = value
            elif name == 'tag':
                [k, v] = re.split(":", value, 1)
                if k=='score':
                    sc_type, rtg = v.split(':')
                    bar_descr[k+'_'+sc_type] = rtg
                else:
                    bar_descr[k] = v
        bar_dict[bar_title] = bar_descr
    return pd.DataFrame(bar_dict).T             

помотреть как выглядят данные можно с помощью команды командной строки (вызывается с префиксом в виде восклицательного знака) head

In [47]:
!head  -5 datasets/RestAll.txt

url:https://www.restoclub.ru/spb/place/le-glamoure/offer/svadba	title:undefined	tag:type:Банкетный зал	tag:subway:undefined	tag:score:Кухня:8.9	tag:score:Интерьер:9.7	tag:score:Сервис:8.9	tag:votes:NaN
url:https://www.restoclub.ru/spb/place/allegro/offer/svadba	title:undefined	tag:type:Банкетный зал	tag:subway:undefined	tag:score:Кухня:9.5	tag:score:Интерьер:9.7	tag:score:Сервис:9.5	tag:votes:NaN
url:https://www.restoclub.ru/spb/place/prospekt/offer/svadba	title:undefined	tag:type:Ресторан	tag:subway:undefined	tag:score:Кухня:9.5	tag:score:Интерьер:9.4	tag:score:Сервис:9.6	tag:votes:NaN
url:https://www.restoclub.ru/spb/place/akvarel-5/offer/svadba	title:undefined	tag:type:Ресторан	tag:subway:undefined	tag:score:Кухня:7.8	tag:score:Интерьер:7.9	tag:score:Сервис:9.2	tag:votes:NaN
url:https://www.restoclub.ru/spb/place/nok/offer/svadba	title:undefined	tag:type:Ресторан	tag:subway:undefined	tag:score:Кухня:8.7	tag:score:Интерьер:9.4	tag:score:Сервис:9.2	tag:votes:NaN


видимо, что это куча строчек одинаково устроенных, поэтому можно разобрать их построчно и отправить в датафрейм, смотрите read_rests_data

In [156]:
filepath ='datasets/RestAll.txt'
df = read_rests_data(filepath)

In [157]:
df.head()

,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes
Земляничные поляны,Ресторан,undefined,1200,0.0,0.0,0.0,3
Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300,0.0,0.0,0.0,1
Che-Dor,Ресторан,undefined,1100,8.6,8.9,4.9,7
Белый аист,Ресторан,Пионерская,1000,9.3,8.2,9.2,8
Мясной цех,Стейк-хаус,Петроградская,2500,7.0,9.0,7.0,1


In [158]:
print(df.info())
print("Тип Object  - общий тип данных, приведем все числовые данные к числам")

<class 'pandas.core.frame.DataFrame'>
Index: 3382 entries, Земляничные поляны to Шашлык-Машлык
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            3382 non-null   object
 1   subway          3382 non-null   object
 2   check           3254 non-null   object
 3   score_Кухня     3382 non-null   object
 4   score_Интерьер  3382 non-null   object
 5   score_Сервис    3382 non-null   object
 6   votes           3382 non-null   object
dtypes: object(7)
memory usage: 211.4+ KB
None
Тип Object  - общий тип данных, приведем все числовые данные к числам


In [159]:
cols = list(df.columns[2:])
print(f'Столбцы, которые хотим привести к числовым {cols}')
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

Столбцы, которые хотим привести к числовым ['check', 'score_Кухня', 'score_Интерьер', 'score_Сервис', 'votes']


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3382 entries, Земляничные поляны to Шашлык-Машлык
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            3382 non-null   object 
 1   subway          3382 non-null   object 
 2   check           3254 non-null   float64
 3   score_Кухня     3381 non-null   float64
 4   score_Интерьер  3381 non-null   float64
 5   score_Сервис    3381 non-null   float64
 6   votes           2413 non-null   float64
dtypes: float64(5), object(2)
memory usage: 211.4+ KB


In [161]:
print('В таблице много пропусков, посмотрим что на них')
df[df.isnull().values]

В таблице много пропусков, посмотрим что на них


,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes
Мама на даче,Ресторан,"Маяковская, Площадь Восстания",1000.0,0.0,0.0,0.0,NaN
Артишок,Ресторан,undefined,1400.0,0.0,0.0,0.0,NaN
Ferma,Кондитерская,Лесная,700.0,0.0,0.0,0.0,NaN
У карьера,Кафе,"Пролетарская, Ломоносовская, Проспект Славы",900.0,0.0,0.0,0.0,NaN
Eggsellent,Корнер,Василеостровская,300.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...
Beer Addicts,Корнер,Пионерская,NaN,0.0,0.0,0.0,NaN
Maklalu,undefined,Петроградская,600.0,0.0,0.0,0.0,NaN
У Борхарда,Ресторан,undefined,1100.0,0.0,0.0,0.0,NaN
Стекло,undefined,Приморская,500.0,0.0,0.0,0.0,NaN


In [162]:
print(f'Просто избавимся от этих данных, останется 2364 заведения')
df = df[df.isnull().values.sum(axis=1)==0]
df

Просто избавимся от этих данных, останется 2364 заведения


,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes
Земляничные поляны,Ресторан,undefined,1200.0,0.0,0.0,0.0,3.0
Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0
Che-Dor,Ресторан,undefined,1100.0,8.6,8.9,4.9,7.0
Белый аист,Ресторан,Пионерская,1000.0,9.3,8.2,9.2,8.0
Мясной цех,Стейк-хаус,Петроградская,2500.0,7.0,9.0,7.0,1.0
...,...,...,...,...,...,...,...
Меншиков,Ресторан,Приморская,700.0,0.0,0.0,0.0,4.0
Охотничья усадьба,Ресторан,undefined,1500.0,0.0,0.0,0.0,3.0
Duplex 2/42,Ресторан,Чкаловская,1100.0,0.0,0.0,0.0,2.0
Король Гамбринус,Ресторан,Ленинский Проспект,900.0,0.0,0.0,0.0,3.0


In [163]:
# Еще вынесем из индекса название и сделаем индекс обычным
df = df.reset_index()
df = df.rename(columns={df.columns[0]: 'Title'})
df.head()

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes
0,Земляничные поляны,Ресторан,undefined,1200.0,0.0,0.0,0.0,3.0
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0
2,Che-Dor,Ресторан,undefined,1100.0,8.6,8.9,4.9,7.0
3,Белый аист,Ресторан,Пионерская,1000.0,9.3,8.2,9.2,8.0
4,Мясной цех,Стейк-хаус,Петроградская,2500.0,7.0,9.0,7.0,1.0


In [165]:
#посмотрим на виды заведений
pd.unique(df.type)

array(['Ресторан', 'Кафе', 'Стейк-хаус', 'Бар', 'Банкетный зал',
       'Ресторанный комплекс', 'Паб', 'Кулинарная студия', 'Пиццерия',
       'Гастробар', 'Корнер', 'Кофейня', 'Бургерная', 'Кальянная',
       'Кейтеринг', 'Винотека', 'undefined', 'Фудмаркет', 'Клуб',
       'Корабль', 'Служба доставки', 'Кондитерская', 'Пекарня', 'Фастфуд',
       'Антикафе', 'Спортбар', 'Лаунж', 'Рюмочная', 'Столовая', 'Лофт',
       'Кулинария', 'Семейный ресторан'], dtype=object)

In [167]:
#А вот метро в таком виде использовать совершенно невозможно
list_subways = pd.unique(df.subway)
all_list = []
[all_list.extend(item.split(',')) for item in list_subways]
all_subways = set([el.strip() for el in all_list])

In [168]:
print(len(all_subways))
centr = set(['Садовая','Сенная Площадь','Невский Проспект','Гостиный Двор','Чернышевская',
             'Маяковская','Владимирская', 'Достоевская', 'Лиговский Проспект', 'Пушкинская', 'Звенигородская'])
petr = set(['Петроградская', 'Чкаловская', 'Горьковская', 'Спортивная'])
VO = set(['Приморская', 'Василеостровская'])
others = all_subways - centr - petr - VO

68


In [188]:
# Вы можете заполнить этот словарь на свое усмотрение
subway_district = {'Centr': centr,
                    'Petr': petr,
                    'VO' : VO,
                    'others': others}

In [173]:
#После этого вам нужно создать столько столбцов, сколько вы сделали районов
keys = list(subway_district.keys())
for k in keys:
    df[f'distr_{k}'] = 0
df.head()

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,distr_Centr,distr_Petr,distr_VO,distr_others
0,Земляничные поляны,Ресторан,undefined,1200.0,0.0,0.0,0.0,3.0,0,0,0,0
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0,0,0,0,0
2,Che-Dor,Ресторан,undefined,1100.0,8.6,8.9,4.9,7.0,0,0,0,0
3,Белый аист,Ресторан,Пионерская,1000.0,9.3,8.2,9.2,8.0,0,0,0,0
4,Мясной цех,Стейк-хаус,Петроградская,2500.0,7.0,9.0,7.0,1.0,0,0,0,0


In [189]:
subway_district['VO']

{'Василеостровская', 'Приморская'}

Дальше надо заполнить эти столбцы, если вы посмтрите там есть некоторые рестораны, для которых вообще разные районы указаны. Будем поступать следующим образом, переберем все записи, и если хоть одна станция метро будет подходить в какой-то район - ставим единичку этому рестрану в колонке с районом


In [185]:
#посмотрите на эту запись
len(set([el.strip() for el in df['subway'].iloc[10].split(',')]).intersection(others))>0

True

In [183]:
for key in list(subway_district.keys()):
     df[f'distr_{key}'] = df.apply(lambda row: 
                                   1*(len(set([el.strip() for el in row['subway'].split(',')]).intersection(subway_district[key]))>0), 
                                   axis=1)

In [184]:
df

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,distr_Centr,distr_Petr,distr_VO,distr_others
0,Земляничные поляны,Ресторан,undefined,1200.0,0.0,0.0,0.0,3.0,0,0,0,1
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0,1,0,0,0
2,Che-Dor,Ресторан,undefined,1100.0,8.6,8.9,4.9,7.0,0,0,0,1
3,Белый аист,Ресторан,Пионерская,1000.0,9.3,8.2,9.2,8.0,0,0,0,1
4,Мясной цех,Стейк-хаус,Петроградская,2500.0,7.0,9.0,7.0,1.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2359,Меншиков,Ресторан,Приморская,700.0,0.0,0.0,0.0,4.0,0,0,1,0
2360,Охотничья усадьба,Ресторан,undefined,1500.0,0.0,0.0,0.0,3.0,0,0,0,1
2361,Duplex 2/42,Ресторан,Чкаловская,1100.0,0.0,0.0,0.0,2.0,0,1,0,0
2362,Король Гамбринус,Ресторан,Ленинский Проспект,900.0,0.0,0.0,0.0,3.0,0,0,0,1


In [191]:
df[df.Title == 'Меншиков']

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,distr_Centr,distr_Petr,distr_VO,distr_others
2359,Меншиков,Ресторан,Приморская,700.0,0.0,0.0,0.0,4.0,0,0,1,0


In [193]:
df_distr = pd.melt(df, id_vars=['Title','type','subway','check','score_Кухня','score_Интерьер','score_Сервис','votes'],
           value_vars=['distr_Centr','distr_Petr','distr_VO','distr_others'], var_name='district')

In [194]:
df_distr.head()

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,district,value
0,Земляничные поляны,Ресторан,undefined,1200.0,0.0,0.0,0.0,3.0,distr_Centr,0
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0,distr_Centr,1
2,Che-Dor,Ресторан,undefined,1100.0,8.6,8.9,4.9,7.0,distr_Centr,0
3,Белый аист,Ресторан,Пионерская,1000.0,9.3,8.2,9.2,8.0,distr_Centr,0
4,Мясной цех,Стейк-хаус,Петроградская,2500.0,7.0,9.0,7.0,1.0,distr_Centr,0


In [195]:
df_distr.shape

(9456, 10)

Получилось что у нас много дублей с нулями, когда заведение может принадлежать к нескольким районам, или сетевой ресторан - имеет смысл с такими данными и работать, но если мы хотим оставить только одну запись на ресторан


In [200]:
df_distr = df_distr[df_distr.value>0].drop(['value'], axis = 1)
df_distr

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,district
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0,distr_Centr
5,Кабинет-портрет,Ресторан,"Владимирская, Достоевская",1200.0,8.6,9.4,8.8,76.0,distr_Centr
6,Legran,Ресторан,"Невский Проспект, Гостиный Двор, Адмиралтейская",2500.0,8.9,9.5,9.0,65.0,distr_Centr
14,Мойка 3,Ресторан,"Невский Проспект, Гостиный Двор",1300.0,8.2,8.4,8.3,4.0,distr_Centr
18,Северянин,Ресторан,"Садовая, Сенная Площадь, Спасская",1500.0,8.1,8.0,8.4,34.0,distr_Centr
...,...,...,...,...,...,...,...,...,...
9449,Mon Ami,Ресторан,Черная Речка,1000.0,0.0,0.0,0.0,1.0,distr_others
9450,Matisov,Ресторан,"Ленинский Проспект, Проспект Ветеранов",1100.0,0.0,0.0,0.0,3.0,distr_others
9452,Охотничья усадьба,Ресторан,undefined,1500.0,0.0,0.0,0.0,3.0,distr_others
9454,Король Гамбринус,Ресторан,Ленинский Проспект,900.0,0.0,0.0,0.0,3.0,distr_others


In [208]:
df_distr.groupby(['district','type'])['check'].agg(['count','mean'])


count         mean
district     type                               
distr_Centr  undefined            3   783.333333
             Антикафе             1   500.000000
             Банкетный зал       44  1869.090909
             Бар                147   738.843537
             Бургерная            5   590.000000
...                             ...          ...
distr_others Служба доставки     11   740.909091
             Спортбар             1   400.000000
             Стейк-хаус           4  1287.500000
             Столовая             1   400.000000
             Фудмаркет            2   550.000000

[91 rows x 2 columns]

In [215]:
#сохраним в табличный формат
df_distr.to_csv('datasets/Restaurants.csv')

In [213]:
#?pd.read_csv

In [217]:
#прочитаем теперь из него
df = pd.read_csv('datasets/Restaurants.csv', index_col= 0)
df

,Title,type,subway,check,score_Кухня,score_Интерьер,score_Сервис,votes,district
1,Лобби-бар отеля «Европа»,Кафе,"Невский Проспект, Гостиный Двор",3300.0,0.0,0.0,0.0,1.0,distr_Centr
5,Кабинет-портрет,Ресторан,"Владимирская, Достоевская",1200.0,8.6,9.4,8.8,76.0,distr_Centr
6,Legran,Ресторан,"Невский Проспект, Гостиный Двор, Адмиралтейская",2500.0,8.9,9.5,9.0,65.0,distr_Centr
14,Мойка 3,Ресторан,"Невский Проспект, Гостиный Двор",1300.0,8.2,8.4,8.3,4.0,distr_Centr
18,Северянин,Ресторан,"Садовая, Сенная Площадь, Спасская",1500.0,8.1,8.0,8.4,34.0,distr_Centr
...,...,...,...,...,...,...,...,...,...
9449,Mon Ami,Ресторан,Черная Речка,1000.0,0.0,0.0,0.0,1.0,distr_others
9450,Matisov,Ресторан,"Ленинский Проспект, Проспект Ветеранов",1100.0,0.0,0.0,0.0,3.0,distr_others
9452,Охотничья усадьба,Ресторан,undefined,1500.0,0.0,0.0,0.0,3.0,distr_others
9454,Король Гамбринус,Ресторан,Ленинский Проспект,900.0,0.0,0.0,0.0,3.0,distr_others
